Prioritize interactions based on most differentially expressed genes and celltypes present in different types of ROIs

In [1]:
import scanpy as sc
import anndata
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from random import randint

In [2]:
adata_wta = sc.read_h5ad(open("/lustre/scratch117/cellgen/team283/Kidney-Nanostring/Kidney_AnnData_4.h5ad", "rb"))

In [3]:
pVals = pd.read_table(open("/lustre/scratch117/cellgen/team283/Kidney-Nanostring/scrna/cellphonedb/out/pvalues.txt"))

In [4]:
columns_split = [c.split('|') for c in pVals.columns]
celltypes = [c for c in columns_split if len(c) == 2]
celltypes_unique = np.unique(np.array(celltypes))
color = []
n = len(celltypes_unique)
for i in range(n):
    color.append('#%06X' % randint(0, 0xFFFFFF))

In [6]:
# Get list of differentially expressed genes:
sc_adata = sc.read_h5ad('/lustre/scratch117/cellgen/team283/Kidney-Nanostring/scrna/mature_adata.h5ad')
de_genes = adata_wta.var.index[[np.abs(adata_wta.varm['DE-CountCorrect-glomerulus_Geometric Segment']['log2fc'].iloc[i]) > 1
and adata_wta.varm['DE-CountCorrect-glomerulus_Geometric Segment']['FDR'].iloc[i] < 0.05
for i in range(len(adata_wta.varm['DE-CountCorrect-glomerulus_Geometric Segment']['FDR']))]]
de_ids = sc_adata.var.loc[de_genes[[d in sc_adata.var.index for d in de_genes]],'ID']

# Make a table that contains all genes that make up a complex:
gene_table = pd.read_csv('/lustre/scratch117/cellgen/team283/Kidney-Nanostring/gene_input.csv')
complex_table = pd.read_csv('/lustre/scratch117/cellgen/team283/Kidney-Nanostring/complex_input.csv')
complex_table = complex_table.iloc[:,0:5]
complex_table['uniprot_1'] = [np.array(gene_table['ensembl'].loc[gene_table['uniprot'] == u])[0] if u in np.array(gene_table['uniprot']) else None for u in complex_table['uniprot_1']]
complex_table['uniprot_2'] = [np.array(gene_table['ensembl'].loc[gene_table['uniprot'] == u])[0] if u in np.array(gene_table['uniprot']) else None for u in complex_table['uniprot_2']]
complex_table['uniprot_3'] = [np.array(gene_table['ensembl'].loc[gene_table['uniprot'] == u])[0] if u in np.array(gene_table['uniprot']) else None for u in complex_table['uniprot_3']]
complex_table['uniprot_4'] = [np.array(gene_table['ensembl'].loc[gene_table['uniprot'] == u])[0] if u in np.array(gene_table['uniprot']) else None for u in complex_table['uniprot_4']]
complex_table.columns = ('complex_name', 'gene_name_1', 'gene_name_2', 'gene_name_3', 'gene_name_4')

# And put this information into the interaction table:
pVals['gene_a'] = [[a] if isinstance(a, str) 
 else list(filter(None,complex_table.loc[:, ('gene_name_1', 'gene_name_2', 'gene_name_3', 'gene_name_4')].iloc[np.where(complex_table['complex_name'] == pVals['partner_a'].iloc[i].split('complex:')[1])[0][0]])) 
 for i,a in enumerate(np.array(pVals['gene_a']))]
pVals['gene_b'] = [[a] if isinstance(a, str) 
 else list(filter(None,complex_table.loc[:, ('gene_name_1', 'gene_name_2', 'gene_name_3', 'gene_name_4')].iloc[np.where(complex_table['complex_name'] == pVals['partner_b'].iloc[i].split('complex:')[1])[0][0]])) 
 for i,a in enumerate(np.array(pVals['gene_b']))]

# Now filter interactions down to those that have at least
# one differentially expressed gene (including any gene in a complex):
de_subset = [np.array([g in np.array(de_ids) for g in pVals['gene_a'].iloc[i]]).any()
 or np.array([g in np.array(de_ids) for g in pVals['gene_b'].iloc[i]]).any()
 for i in range(len(pVals['gene_a']))]
pVals_subset = pVals.loc[de_subset,:]

# Also subset to most abundant cell type in ROI type of interest:
cutoff = 100
split_columns = [o.split('q05_cell_density_per_mm2_') for o in adata_wta.obs.columns]
celltypes_columns = [s[1]  for s in split_columns if len(s) == 2]
celldensity = adata_wta.obs.loc[:,[len(s) == 2 for s in split_columns]]
celltypes_columns == celltypes_unique
roi_subset = np.array([adata_wta.obs['region'].iloc[i] + adata_wta.obs['SegmentLabel'].iloc[i]
for i in range(len(adata_wta.obs['region']))]) == 'glomerulusGeometric Segment'
mean_density = np.mean(celldensity.loc[roi_subset,:], axis = 0)
mean_density.index = celltypes_columns
celltypes_subset = mean_density.index[mean_density > cutoff]

# And make sankey the plots just for this subset of genes and cell types:
columns_split = [c.split('|') for c in pVals_subset.columns]
celltypes = [c for c in columns_split if len(c) == 2]
celltypes_unique = np.unique(np.array(celltypes))
label = np.concatenate([[c + ' (ligand)' for c in celltypes_unique], [c + ' (receptor)' for c in celltypes_unique]])
count = 0
source = []
target = []
value = []
for i in range(11, np.shape(pVals_subset)[1]):
        source.append(np.where(celltypes_unique == celltypes[count][0])[0][0])
        target.append(np.where(celltypes_unique == celltypes[count][1])[0][0] + 33)
        value.append(np.sum([pVals_subset.iloc[j,i] < 0.05 and pVals_subset.loc[:,'receptor_b'].iloc[j]
        for j in range(len(pVals_subset.loc[:,'receptor_b']))]))
        
        source.append(np.where(celltypes_unique == celltypes[count][1])[0][0])
        target.append(np.where(celltypes_unique == celltypes[count][0])[0][0] + 33)
        value.append(np.sum([pVals_subset.iloc[j,i] < 0.05 and pVals_subset.loc[:,'receptor_a'].iloc[j]
        for j in range(len(pVals_subset.loc[:,'receptor_a']))]))
        
        count += 1          
celltypes_subset_index = [np.where(celltypes_unique == c)[0][0] for c in celltypes_subset]
subset = [source[i] in np.array(celltypes_subset_index)
          and target[i]-33 in np.array(celltypes_subset_index)
          for i in range(len(source))]
source_subset = np.array(source)[subset]
target_subset = np.array(target)[subset]
value_subset = np.array(value)[subset]
fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = label,
      color = np.concatenate([color, color])
    ),
    link = dict(
      source = source_subset, # indices correspond to labels, eg A1, A2, A1, B1, ...
      target = target_subset,
      value = value_subset
  ))])

fig.update_layout(title_text="CellphoneDB Diagram Glomeruli", font_size=10)
fig.show()
fig.write_html("../sankey.html")

# Also save prioritized interactions in table format:
columns_split = [c.split('|') for c in pVals_subset.columns]
celltypes = [c for c in columns_split if len(c) == 2]
pVals_prioritized = pVals.loc[:,np.concatenate([np.repeat(True, 11), [c[0] in celltypes_subset and c[1] in celltypes_subset for c in celltypes]])]

Make a table with detailed information about all prioritized interactions:

In [60]:
prioritizedInteractions = pd.DataFrame(columns = ['Celltype (ligand)', 'Celltype (receptor)', 'Ligand Protein', 'Receptor Protein', 'Ligand Genes', 'Receptor Genes',
                                                  'Ligand Genes log2FC', 'Receptor Genes log2FC', 'Ligand Genes P-Value',
                                                  'Receptor Genes P-Value', 'Ligand+Receptor Genes Mean log2FC'])

Also make a simple version by just subsetting pVals: